In [1]:
import cProfile
import lysis
import math
import numpy as np

In [2]:
e = lysis.util.Experiment(r'../../data', experiment_code='2022-12-27-1100')

In [3]:
p = {'total_time': 1}
e.initialize_macro_param(p)
# e.read_file()

In [4]:
macro = lysis.MacroscaleRun(e)

In [5]:
macro.run()

100%|█████████████████████████████████████████████████████████████████████████████| 2920/2920 [00:05<00:00, 523.12it/s]

Total binds: 11,029
Timesteps with changes to degrade time: 723
Total moves: 21,196,167
Total macro unbinds: 0
Total micro unbinds: 19


In [7]:
current_time = 2920 * macro.exp.macro_params.time_step
current_time

0.9997907233333332

In [9]:
macro.m_fiber_status = macro.fiber_status[macro.location]
macro.unbind_by_degradation(macro.bound & (macro.m_fiber_status < current_time), current_time)
macro.unbind_by_time(macro.bound & (macro.binding_time < current_time), current_time)

should_bind = (~macro.bound
               & (macro.binding_time < current_time)
               & (macro.m_fiber_status > current_time)
               & (macro.waiting_time < current_time))

move_chance = np.ones(macro.exp.macro_params.total_molecules, dtype=np.double)
move_chance[~macro.bound] = macro.rng.random(np.count_nonzero(~macro.bound), dtype=np.double)
should_move = move_chance < macro.exp.macro_params.moving_probability
conflict = should_bind & should_move
threshold = np.full(macro.exp.macro_params.total_molecules, -1, dtype=np.double)
threshold[conflict] = ((current_time - macro.binding_time[conflict])
                       / macro.exp.macro_params.time_step)
should_bind[conflict] = macro.rng.random(np.count_nonzero(conflict)) > threshold[conflict]
should_move[conflict] = ~should_bind[conflict]

macro.bind(should_bind, current_time)

In [10]:
still_stuck_to_fiber = np.full(macro.exp.macro_params.total_molecules, False, dtype=np.bool_)
still_stuck_to_fiber[should_bind] = macro.waiting_time[should_bind] > current_time
np.logical_and(still_stuck_to_fiber[should_bind], macro.unbound_by_degradation[should_bind], out=still_stuck_to_fiber[should_bind])
np.count_nonzero(still_stuck_to_fiber)

0

In [11]:
for n in np.arange(macro.exp.macro_params.total_molecules)[still_stuck_to_fiber]:
    print(n)

In [15]:
a = 499
b = 500
c = a*e.macro_params.time_step
d = b*e.macro_params.time_step

In [16]:
%timeit a < b

36.1 ns ± 0.269 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [17]:
%timeit c < d

33.1 ns ± 0.401 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [ ]:
lysis.from_fortran_edge_index(51, 4, 5)

In [15]:
rng = np.random.default_rng(seed=489489496)

In [57]:
a = np.arange(60).reshape(20,3)

In [70]:
b = rng.random(20) > 0.5
c = rng.integers(3, size=(np.count_nonzero(b),))
a[b, c]

array([ 3,  7, 19, 22, 26, 29, 30, 38, 46, 56, 57])

In [71]:
a, b, c

(array([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17],
        [18, 19, 20],
        [21, 22, 23],
        [24, 25, 26],
        [27, 28, 29],
        [30, 31, 32],
        [33, 34, 35],
        [36, 37, 38],
        [39, 40, 41],
        [42, 43, 44],
        [45, 46, 47],
        [48, 49, 50],
        [51, 52, 53],
        [54, 55, 56],
        [57, 58, 59]]),
 array([False,  True,  True, False, False, False,  True,  True,  True,
         True,  True, False,  True, False, False,  True, False, False,
         True,  True]),
 array([0, 1, 1, 1, 2, 2, 0, 2, 1, 2, 0], dtype=int64))

In [72]:
a[b]

array([[ 3,  4,  5],
       [ 6,  7,  8],
       [18, 19, 20],
       [21, 22, 23],
       [24, 25, 26],
       [27, 28, 29],
       [30, 31, 32],
       [36, 37, 38],
       [45, 46, 47],
       [54, 55, 56],
       [57, 58, 59]])

In [76]:
a = rng.random(20000)

In [80]:
%timeit np.count_nonzero(a < 0.1)

4.84 µs ± 64.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [81]:
%timeit np.any(a < 0.1)

8.26 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [83]:
a = np.empty((20, 9), dtype=int)

In [84]:
a[:] = np.arange(9)

In [85]:
a

array([[0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8]])

In [86]:
a[4,6]=100
a

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5, 100,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3, 